In [2]:
import os
import pandas as pd
import sys
import numpy as np
import sys
import re
sys.path.insert(0, '../..')
import itertools

from genepy.utils import helper as h
from genepy.epigenetics import chipseq as chip
from genepy.epigenetics import plot as genepyPlot

import dalmatian as dm
import pyBigWig

from bokeh.plotting import *
from IPython.display import IFrame
import igv
from biomart import BiomartServer
import io

from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture

output_notebook()
%load_ext autoreload
%autoreload 2

Loading BokehJS ...

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# ChIP

## IRF2BP2_degraded_v6

In [2]:
! gsutil ls gs://amlproject/Chip/

gs://amlproject/Chip/GEO/
gs://amlproject/Chip/IRF2BP2_degraded_v1/
gs://amlproject/Chip/IRF2BP2_degraded_v2/
gs://amlproject/Chip/IRF2BP2_degraded_v3/
gs://amlproject/Chip/IRF2BP2_degraded_v4/
gs://amlproject/Chip/IRF2BP2_degraded_v5/
gs://amlproject/Chip/IRF2BP2_degraded_v6/
gs://amlproject/Chip/IRF8_degraded_v1/
gs://amlproject/Chip/MEF2CMEF2D_knockout_v1/
gs://amlproject/Chip/MEF2D_degraded_v1/
gs://amlproject/Chip/MEF2D_degraded_v2/
gs://amlproject/Chip/MEF2D_degraded_v3/
gs://amlproject/Chip/MEF2D_degraded_v4/
gs://amlproject/Chip/cobinding/
gs://amlproject/Chip/fastqs/
gs://amlproject/Chip/knockouts_v1/


In [3]:
project = "IRF2BP2_degraded_v6"
loc = "210331_MP8777_fastq" # transfer AML folder

In [4]:
fastq = ! gsutil ls gs://transfer-amlproject/$loc/*.fastq.gz

for f in fastq:
    print(os.path.basename(f))

20210331_DMSO_FLAG_A_MP8777_S94_R1_001.fastq.gz
20210331_DMSO_FLAG_A_MP8777_S94_R2_001.fastq.gz
20210331_DMSO_FLAG_B_MP8777_S95_R1_001.fastq.gz
20210331_DMSO_FLAG_B_MP8777_S95_R2_001.fastq.gz
20210331_DMSO_H3K27ac_A_MP8777_S98_R1_001.fastq.gz
20210331_DMSO_H3K27ac_A_MP8777_S98_R2_001.fastq.gz
20210331_DMSO_H3K27ac_B_MP8777_S99_R1_001.fastq.gz
20210331_DMSO_H3K27ac_B_MP8777_S99_R2_001.fastq.gz
20210331_DMSO_MED1_A_MP8777_S96_R1_001.fastq.gz
20210331_DMSO_MED1_A_MP8777_S96_R2_001.fastq.gz
20210331_DMSO_MED1_B_MP8777_S97_R1_001.fastq.gz
20210331_DMSO_MED1_B_MP8777_S97_R2_001.fastq.gz
20210331_VHL_FLAG_A_MP8777_S100_R1_001.fastq.gz
20210331_VHL_FLAG_A_MP8777_S100_R2_001.fastq.gz
20210331_VHL_FLAG_B_MP8777_S101_R1_001.fastq.gz
20210331_VHL_FLAG_B_MP8777_S101_R2_001.fastq.gz
20210331_VHL_H3K27ac_A_MP8777_S104_R1_001.fastq.gz
20210331_VHL_H3K27ac_A_MP8777_S104_R2_001.fastq.gz
20210331_VHL_H3K27ac_B_MP8777_S105_R1_001.fastq.gz
20210331_VHL_H3K27ac_B_MP8777_S105_R2_001.fastq.gz
20210331_VHL_MED

In [27]:
mkdir ../data/$project/ && mkdir ../data/$project/qc/ #make dir in /home/monika/AMLproject/data

In [29]:
! gsutil -m cp gs://transfer-amlproject/$loc/multiqc_report.html ../data/$project/qc/
! gsutil -m cp -r gs://transfer-amlproject/$loc/Reports/ ../data/$project/qc/
! gsutil -m cp -r gs://transfer-amlproject/$loc/multiqc_data/ ../data/$project/qc/

Copying gs://transfer-amlproject/210331_MP8777_fastq/multiqc_report.html...
/ [1/1 files][  1.4 MiB/  1.4 MiB] 100% Done                                    
Operation completed over 1 objects/1.4 MiB.                                      
Copying gs://transfer-amlproject/210331_MP8777_fastq/Reports/20210331_DMSO_FLAG_A_MP8777_S94_R1_001_fastqc.html...
Copying gs://transfer-amlproject/210331_MP8777_fastq/Reports/20210331_DMSO_FLAG_A_MP8777_S94_R1_001_fastqc.zip...
Copying gs://transfer-amlproject/210331_MP8777_fastq/Reports/20210331_DMSO_FLAG_A_MP8777_S94_R2_001_fastqc.html...
Copying gs://transfer-amlproject/210331_MP8777_fastq/Reports/20210331_DMSO_FLAG_A_MP8777_S94_R2_001_fastqc.zip...
Copying gs://transfer-amlproject/210331_MP8777_fastq/Reports/20210331_DMSO_FLAG_B_MP8777_S95_R1_001_fastqc.html...
Copying gs://transfer-amlproject/210331_MP8777_fastq/Reports/20210331_DMSO_FLAG_B_MP8777_S95_R1_001_fastqc.zip...
Copying gs://transfer-amlproject/210331_MP8777_fastq/Reports/20210331_DMSO

In [30]:
! gsutil -m cp gs://transfer-amlproject/$loc/*  gs://transfer-amlproject/$project/

Omitting prefix "gs://transfer-amlproject/210331_MP8777_fastq/Reports/". (Did you mean to do cp -r?)
Omitting prefix "gs://transfer-amlproject/210331_MP8777_fastq/multiqc_data/". (Did you mean to do cp -r?)
Copying gs://transfer-amlproject/210331_MP8777_fastq/20210331_DMSO_FLAG_A_MP8777_S94_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/210331_MP8777_fastq/20210331_DMSO_FLAG_A_MP8777_S94_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/210331_MP8777_fastq/20210331_DMSO_FLAG_B_MP8777_S95_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/210331_MP8777_fastq/20210331_DMSO_FLAG_B_MP8777_S95_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/210331_MP8777_fastq/20210331_DMSO_H3K27ac_A_MP8777_S98_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/210331_MP8777_fastq/20210331_DMSO_H3K

### analysis

In [6]:
#! mkdir ../../data/$project && mkdir ../../data/$project/fastqs \
#&& gsutil -m cp gs://transfer-amlproject/$project/* ../../data/$project/fastqs

Copying gs://transfer-amlproject/IRF2BP2_degraded_v6/20210331_DMSO_FLAG_A_MP8777_S94_R1_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_degraded_v6/20210331_DMSO_FLAG_A_MP8777_S94_R2_001.fastq.gz...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://transfer-amlproject/IRF2BP2_degraded_v6/20210331_DMSO_FLAG_B_MP8777_S95_R1_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_degraded_v6/20210331_DMSO_FLAG_B_MP8777_S95_R2_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_degraded_v6/20210331_DMSO_H3K27ac_A_MP8777_S98_R1_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_degraded_v6/20210331_DMSO_H3K27ac_A_MP8777_S98_R2_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_degraded_v6/20210331_DMSO_H3K27ac_B_MP8777_S99_R1_001.fastq.gz...
Copyin

In [25]:
! gsutil -m cp -r gs://amlproject/Chip/$project/fastqs ../../data/$project/fastqs

Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/fastqs/mp967-MV411_IRF2BP2_DMSO-FLAG_IRF2BP2-r1_R1_001.fastq.gz...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/fastqs/mp969-MV411_IRF2BP2_DMSO-FLAG_IRF2BP2-r2_R1_001.fastq.gz...
Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/fastqs/mp967-MV411_IRF2BP2_DMSO-FLAG_IRF2BP2-r1_R2_001.fastq.gz...
Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/fastqs/mp969-MV411_IRF2BP2_DMSO-FLAG_IRF2BP2-r2_R2_001.fastq.gz...
Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/fastqs/mp971-MV411_IRF2BP2_DMSO-H3K27AC-r1_R1_001.fastq.gz...
Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/fastqs/mp971-MV411_IRF2BP2_DMSO-H3K27AC-r1_R2_001.fastq.gz...
Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/fastqs/m

In [5]:
fastq_name_list = [i.split("/")[4].split(".")[0] for i in fastq]
len(fastq_name_list)
fastq_name_list

['20210331_DMSO_FLAG_A_MP8777_S94_R1_001',
 '20210331_DMSO_FLAG_A_MP8777_S94_R2_001',
 '20210331_DMSO_FLAG_B_MP8777_S95_R1_001',
 '20210331_DMSO_FLAG_B_MP8777_S95_R2_001',
 '20210331_DMSO_H3K27ac_A_MP8777_S98_R1_001',
 '20210331_DMSO_H3K27ac_A_MP8777_S98_R2_001',
 '20210331_DMSO_H3K27ac_B_MP8777_S99_R1_001',
 '20210331_DMSO_H3K27ac_B_MP8777_S99_R2_001',
 '20210331_DMSO_MED1_A_MP8777_S96_R1_001',
 '20210331_DMSO_MED1_A_MP8777_S96_R2_001',
 '20210331_DMSO_MED1_B_MP8777_S97_R1_001',
 '20210331_DMSO_MED1_B_MP8777_S97_R2_001',
 '20210331_VHL_FLAG_A_MP8777_S100_R1_001',
 '20210331_VHL_FLAG_A_MP8777_S100_R2_001',
 '20210331_VHL_FLAG_B_MP8777_S101_R1_001',
 '20210331_VHL_FLAG_B_MP8777_S101_R2_001',
 '20210331_VHL_H3K27ac_A_MP8777_S104_R1_001',
 '20210331_VHL_H3K27ac_A_MP8777_S104_R2_001',
 '20210331_VHL_H3K27ac_B_MP8777_S105_R1_001',
 '20210331_VHL_H3K27ac_B_MP8777_S105_R2_001',
 '20210331_VHL_MED1_A_MP8777_S102_R1_001',
 '20210331_VHL_MED1_A_MP8777_S102_R2_001',
 '20210331_VHL_MED1_B_MP8777_S

In [7]:
# accidentally named mp982 & not mp981 which would be consistent with other samples
rename = {
    '20210331_DMSO_FLAG_A_MP8777_S94': "mp967-MV411_IRF2BP2_DMSO-FLAG_IRF2BP2-r1",
    '20210331_DMSO_FLAG_B_MP8777_S95': "mp969-MV411_IRF2BP2_DMSO-FLAG_IRF2BP2-r2",
    '20210331_DMSO_H3K27ac_A_MP8777_S98': "mp971-MV411_IRF2BP2_DMSO-H3K27AC-r1",
    '20210331_DMSO_H3K27ac_B_MP8777_S99': "mp973-MV411_IRF2BP2_DMSO-H3K27AC-r2",
    '20210331_DMSO_MED1_A_MP8777_S96': "mp975-MV411_IRF2BP2_DMSO-MED1-r1",
    '20210331_DMSO_MED1_B_MP8777_S97': "mp977-MV411_IRF2BP2_DMSO-MED1-r2",
    
    '20210331_VHL_FLAG_A_MP8777_S100': "mp979-MV411_IRF2BP2_VHL-FLAG_IRF2BP2-r1",
    '20210331_VHL_FLAG_B_MP8777_S101': "mp982-MV411_IRF2BP2_VHL-FLAG_IRF2BP2-r2", # naming mistake
    '20210331_VHL_H3K27ac_A_MP8777_S104': "mp983-MV411_IRF2BP2_VHL-H3K27AC-r1",
    '20210331_VHL_H3K27ac_B_MP8777_S105': "mp985-MV411_IRF2BP2_VHL-H3K27AC-r2",
    '20210331_VHL_MED1_A_MP8777_S102': "mp987-MV411_IRF2BP2_VHL-MED1-r1",
    '20210331_VHL_MED1_B_MP8777_S103': "mp989-MV411_IRF2BP2_VHL-MED1-r2"
}

In [8]:
# rename fastqs and copy to amlproject
for val in fastq:
    rep = val
    for k,v in rename.items():
        rep = rep.replace(k,v)
    rep = rep.replace('transfer-amlproject/'+loc+'/',"amlproject/Chip/"+project+'/fastqs/')
    print(rep)
    #! gsutil cp $val $rep

gs://amlproject/Chip/IRF2BP2_degraded_v6/fastqs/mp967-MV411_IRF2BP2_DMSO-FLAG_IRF2BP2-r1_R1_001.fastq.gz
gs://amlproject/Chip/IRF2BP2_degraded_v6/fastqs/mp967-MV411_IRF2BP2_DMSO-FLAG_IRF2BP2-r1_R2_001.fastq.gz
gs://amlproject/Chip/IRF2BP2_degraded_v6/fastqs/mp969-MV411_IRF2BP2_DMSO-FLAG_IRF2BP2-r2_R1_001.fastq.gz
gs://amlproject/Chip/IRF2BP2_degraded_v6/fastqs/mp969-MV411_IRF2BP2_DMSO-FLAG_IRF2BP2-r2_R2_001.fastq.gz
gs://amlproject/Chip/IRF2BP2_degraded_v6/fastqs/mp971-MV411_IRF2BP2_DMSO-H3K27AC-r1_R1_001.fastq.gz
gs://amlproject/Chip/IRF2BP2_degraded_v6/fastqs/mp971-MV411_IRF2BP2_DMSO-H3K27AC-r1_R2_001.fastq.gz
gs://amlproject/Chip/IRF2BP2_degraded_v6/fastqs/mp973-MV411_IRF2BP2_DMSO-H3K27AC-r2_R1_001.fastq.gz
gs://amlproject/Chip/IRF2BP2_degraded_v6/fastqs/mp973-MV411_IRF2BP2_DMSO-H3K27AC-r2_R2_001.fastq.gz
gs://amlproject/Chip/IRF2BP2_degraded_v6/fastqs/mp975-MV411_IRF2BP2_DMSO-MED1-r1_R1_001.fastq.gz
gs://amlproject/Chip/IRF2BP2_degraded_v6/fastqs/mp975-MV411_IRF2BP2_DMSO-MED1-r1_R2

In [9]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret_web.json', '~/.storage.json')
url = "https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U/edit?ts=5fab1071#gid=738732237"
gsheet = sheets.get(url).sheets[3].to_frame()

In [111]:
#! gsutil rm gs://amlproject/Chip/$project/fastqs/multiqc_report.html

CommandException: No URLs matched: gs://amlproject/Chip/IRF2BP2_degraded_v6/fastqs/multiqc_report.html


In [10]:
fastq = ! gsutil ls gs://amlproject/Chip/$project/fastqs/

In [12]:
[os.path.basename(f) for f in fastq]

['mp967-MV411_IRF2BP2_DMSO-FLAG_IRF2BP2-r1_R1_001.fastq.gz',
 'mp967-MV411_IRF2BP2_DMSO-FLAG_IRF2BP2-r1_R2_001.fastq.gz',
 'mp969-MV411_IRF2BP2_DMSO-FLAG_IRF2BP2-r2_R1_001.fastq.gz',
 'mp969-MV411_IRF2BP2_DMSO-FLAG_IRF2BP2-r2_R2_001.fastq.gz',
 'mp971-MV411_IRF2BP2_DMSO-H3K27AC-r1_R1_001.fastq.gz',
 'mp971-MV411_IRF2BP2_DMSO-H3K27AC-r1_R2_001.fastq.gz',
 'mp973-MV411_IRF2BP2_DMSO-H3K27AC-r2_R1_001.fastq.gz',
 'mp973-MV411_IRF2BP2_DMSO-H3K27AC-r2_R2_001.fastq.gz',
 'mp975-MV411_IRF2BP2_DMSO-MED1-r1_R1_001.fastq.gz',
 'mp975-MV411_IRF2BP2_DMSO-MED1-r1_R2_001.fastq.gz',
 'mp977-MV411_IRF2BP2_DMSO-MED1-r2_R1_001.fastq.gz',
 'mp977-MV411_IRF2BP2_DMSO-MED1-r2_R2_001.fastq.gz',
 'mp979-MV411_IRF2BP2_VHL-FLAG_IRF2BP2-r1_R1_001.fastq.gz',
 'mp979-MV411_IRF2BP2_VHL-FLAG_IRF2BP2-r1_R2_001.fastq.gz',
 'mp982-MV411_IRF2BP2_VHL-FLAG_IRF2BP2-r2_R1_001.fastq.gz',
 'mp982-MV411_IRF2BP2_VHL-FLAG_IRF2BP2-r2_R2_001.fastq.gz',
 'mp983-MV411_IRF2BP2_VHL-H3K27AC-r1_R1_001.fastq.gz',
 'mp983-MV411_IRF2BP2_VHL

In [13]:
# design matrix
df = {
"fastq_1": [],
"fastq_2": [],
"antibody": [],
"group": [],
"replicate": [],
"control": []
}
for val in h.grouped(fastq, 2):
    row = gsheet[gsheet.id==val[0].split('/')[-1].split('-')[0]]
    df['group'].append("MV4"+row['name'].values[0].split('-r')[0].split('-MV4')[1])
    df['replicate'].append(row['name'].values[0].split('-r')[1])
    df['fastq_1'].append(val[0])
    df['fastq_2'].append(val[1])
    df['antibody'].append(row['protein'].values[0])
    df['control'].append("INPUT")
df['group'].append('INPUT')
df['replicate'].append(1)
df['fastq_1'].append('gs://amlproject/Chip/fastqs/paired_end/mp845-MV411-INPUT-r2_R1.fastq.gz')
df['fastq_2'].append('gs://amlproject/Chip/fastqs/paired_end/mp845-MV411-INPUT-r2_R2.fastq.gz')
df['antibody'].append("")
df['control'].append("")
df = pd.DataFrame(df)

In [14]:
# nextflow CSV
design_file = "../nextflow/"+project+"_design.csv"
design_file_path = os.path.abspath(design_file)
print(design_file_path)
#df[df.columns[[3,4,0,1,2,5]]].to_csv(design_file_path,index=False)

/home/monika/AMLproject/nextflow/IRF2BP2_degraded_v6_design.csv


In [17]:
df = pd.read_csv(design_file) # read in design file
df = df.replace(np.nan, '', regex=True)
df = df[["fastq_1", "fastq_2", "antibody", "group", "replicate", "control"]]
df.iloc[:2,:]

,fastq_1,fastq_2,antibody,group,replicate,control
0,gs://amlproject/Chip/IRF2BP2_degraded_v6/fastq...,gs://amlproject/Chip/IRF2BP2_degraded_v6/fastq...,FLAG_IRF2BP2,MV411_IRF2BP2_DMSO-FLAG_IRF2BP2,1,INPUT
1,gs://amlproject/Chip/IRF2BP2_degraded_v6/fastq...,gs://amlproject/Chip/IRF2BP2_degraded_v6/fastq...,FLAG_IRF2BP2,MV411_IRF2BP2_DMSO-FLAG_IRF2BP2,2,INPUT


### about

0. you need to have a google project set up with a billing account
1. you need to activte your APIs this way: https://cloud.google.com/life-sciences/docs/tutorials/nextflow?hl=fr
2. nextflow needs to be installed with this installation command `export NXF_MODE=google && curl https://get.nextflow.io | bash`

In [35]:
! cd /home/monika && sudo ./nextflow log ## to get access to the previous runs

TIMESTAMP          	DURATION  	RUN NAME         	STATUS	REVISION ID	SESSION ID                          	COMMAND                                                                                                                                                                                                                                                                                                                                                                                                          
2021-07-08 20:08:32	-         	hopeful_albattani	-     	fb4924dd0d 	117e88ad-5b5f-4ad3-a368-4ebee2f804c8	nextflow run monikaperez/chipseq --paired_end --seq_center DFCI --email jkobject@gmail.com --input AMLproject/nextflow/MEF2D_degraded_v4_design.csv --genome GRCh38 --max_cpus 16 -profile mpcloud -w 'gs://workamlproject/MEF2Cv4res' --spiking --skip_diff_analysis --outdir 'gs://workamlproject/MEF2Cv4' --tracedir AMLproject/nextflow/MEF2Cdegradedv4/                                        

In [18]:
# write nextflow command to string
export_header = "export NXF_mode=google && sudo"
nextflow_call = "./nextflow run monikaperez/chipseq" # run outside of AMLproject dir
endedness = "--paired_end"
seq_center = "--seq_center " + "'DFCI'"
email = "--email " + "'monikawenperez@gmail.com'"
design_input = "--input " + design_file_path
genome = "--genome " + "GRCh38"
max_cpus = "--max_cpus " + str(16)
profile = "-profile " + "mpcloud"
work_dir = "-w " + "gs://workamlproject/{}res".format(project)
outdir = "--outdir "+"gs://workamlproject/"+project
spiking = "--spiking"
tracedir = "--tracedir AMLproject/nextflow/"+project
#peak_type = "--narrow_peak"

nextflow_args = [export_header, nextflow_call, endedness,
                 seq_center, email, design_input, 
                 genome, max_cpus, profile, 
                 work_dir, outdir, spiking, tracedir,
                 "--skip_diff_analysis"]

# process chips
print("Copy and paste string into command line:\n")
print("\n".join(nextflow_args))

Copy and paste string into command line:

export NXF_mode=google && sudo
./nextflow run monikaperez/chipseq
--paired_end
--seq_center 'DFCI'
--email 'monikawenperez@gmail.com'
--input /home/monika/AMLproject/nextflow/IRF2BP2_degraded_v6_design.csv
--genome GRCh38
--max_cpus 16
-profile mpcloud
-w gs://workamlproject/IRF2BP2_degraded_v6res
--outdir gs://workamlproject/IRF2BP2_degraded_v6
--spiking
--tracedir AMLproject/nextflow/IRF2BP2_degraded_v6
--skip_diff_analysis


In [91]:
! gsutil -m cp -r gs://workamlproject/$project/* gs://amlproject/Chip/$project/ # move results to main bucket

Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam.bai [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1.mLb.clN.sorted.bam [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1.mLb.clN.sorted.bam.bai [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2.mLb.clN.sorted.bam [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2.mLb.clN.sorted.bam.bai [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibr

Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/counts/INPUT_R1_T1.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/counts/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_T1.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/counts/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_T1.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/counts/MV411_IRF2BP2_DMSO-H3K27AC_R1_T1.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/counts/MV411_IRF2BP2_DMSO-H3K27AC_R2_T1.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/counts/MV411_IRF2BP2_DMSO-MED1_R1_T1.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/counts/MV411_IRF2BP2_DMSO-MED1

Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_IRF2BP2_VHL-MED1_R2.plotFingerprint.raw.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/deepTools/plotProfile/INPUT_R1.computeMatrix.mat.gz [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/deepTools/plotProfile/INPUT_R1.computeMatrix.vals.mat.tab [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/deepTools/plotProfile/INPUT_R1.plotHeatmap.mat.tab [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/deepTools/plotProfile/INPUT_R1.plotHeatmap.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/deepTools/plotProfile/INPUT_R1.plotProfile.pdf [Content-Type=application/octet-stream]...
Copying gs://wo

Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/deepTools/plotProfile/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1.plotHeatmap.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/deepTools/plotProfile/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1.plotProfile.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/deepTools/plotProfile/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1.plotProfile.tab [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/deepTools/plotProfile/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2.computeMatrix.mat.gz [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/deepTools/plotProfile/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2.computeMatrix.vals.mat.tab [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/deepTools/plo

Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/MV411_IRF2BP2_DMSO-H3K27AC_R2_peaks.gappedPeak [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/MV411_IRF2BP2_DMSO-H3K27AC_R2_peaks.xls [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/MV411_IRF2BP2_DMSO-MED1_R1_peaks.annotatePeaks.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/MV411_IRF2BP2_DMSO-MED1_R1_peaks.broadPeak [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/MV411_IRF2BP2_DMSO-MED1_R1_peaks.gappedPeak [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/MV411_IRF2BP2_DMSO-MED1_R1_peaks.xls [Content-Type=application/octet-strea

...

Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/samtools_stats/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2.mLb.mkD.sorted.bam.idxstats [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/samtools_stats/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2.mLb.mkD.sorted.bam.stats [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/samtools_stats/MV411_IRF2BP2_DMSO-H3K27AC_R1.mLb.clN.sorted.bam.flagstat [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/samtools_stats/MV411_IRF2BP2_DMSO-H3K27AC_R1.mLb.clN.sorted.bam.idxstats [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/samtools_stats/MV411_IRF2BP2_DMSO-H3K27AC_R1.mLb.clN.sorted.bam.stats [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/samtools_stats/MV411

Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/samtools_stats/MV411_IRF2BP2_VHL-H3K27AC_R2.mLb.clN.sorted.bam.stats [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/samtools_stats/MV411_IRF2BP2_VHL-H3K27AC_R2.mLb.mkD.sorted.bam.flagstat [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/samtools_stats/MV411_IRF2BP2_VHL-H3K27AC_R2.mLb.mkD.sorted.bam.idxstats [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/samtools_stats/MV411_IRF2BP2_VHL-H3K27AC_R2.mLb.mkD.sorted.bam.stats [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/samtools_stats/MV411_IRF2BP2_VHL-MED1_R1.mLb.clN.sorted.bam.flagstat [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/bwa/mergedLibrary/samtools_stats/MV411_IRF2BP2_VHL-MED1_

Copying gs://workamlproject/IRF2BP2_degraded_v6/droso_aligned/scale/MV411_IRF2BP2_VHL-H3K27AC_R2_T1_scaling.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/droso_aligned/scale/MV411_IRF2BP2_VHL-MED1_R1_T1_scaling.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/droso_aligned/scale/MV411_IRF2BP2_VHL-MED1_R2_T1_scaling.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/fastqc/INPUT_R1_T1_1_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/fastqc/INPUT_R1_T1_2_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/fastqc/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_T1_1_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/fastqc/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_T1_2_fastqc.html [Content-Type=application/octet-stre

Copying gs://workamlproject/IRF2BP2_degraded_v6/genome/genome.fa.include_regions.bed [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/genome/genome.fa.sizes [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/igv/broadPeak/igv_files.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/igv/broadPeak/igv_session.xml [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/multiqc/broadPeak/multiqc_report.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/pipeline_info/design_controls.csv [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/pipeline_info/design_reads.csv [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/pipeline_info/results_description.html [Content-Type=application/octet-stream]...
Copying gs:/

Copying gs://workamlproject/IRF2BP2_degraded_v6/trim_galore/fastqc/zips/MV411_IRF2BP2_VHL-H3K27AC_R1_T1_2_val_2_fastqc.zip [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/trim_galore/fastqc/zips/MV411_IRF2BP2_VHL-H3K27AC_R2_T1_1_val_1_fastqc.zip [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/trim_galore/fastqc/zips/MV411_IRF2BP2_VHL-H3K27AC_R2_T1_2_val_2_fastqc.zip [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/trim_galore/fastqc/zips/MV411_IRF2BP2_VHL-MED1_R1_T1_1_val_1_fastqc.zip [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/trim_galore/fastqc/zips/MV411_IRF2BP2_VHL-MED1_R1_T1_2_val_2_fastqc.zip [Content-Type=application/octet-stream]...
Copying gs://workamlproject/IRF2BP2_degraded_v6/trim_galore/fastqc/zips/MV411_IRF2BP2_VHL-MED1_R2_T1_1_val_1_fastqc.zip [Content-Type=application/octet-stream]...
Copying gs://

In [93]:
! gsutil cat gs://amlproject/Chip/$project/droso_aligned/counts/* # get unique mapped reads(droso)
! gsutil ls gs://amlproject/Chip/$project/droso_aligned/counts/

2923704
1313814
1475226
2680444
1429718
721324
1444440
1713190
1024296
2024650
1891418
1285272
1107752
gs://amlproject/Chip/IRF2BP2_degraded_v6/droso_aligned/counts/INPUT_R1_T1_counts.txt
gs://amlproject/Chip/IRF2BP2_degraded_v6/droso_aligned/counts/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_T1_counts.txt
gs://amlproject/Chip/IRF2BP2_degraded_v6/droso_aligned/counts/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_T1_counts.txt
gs://amlproject/Chip/IRF2BP2_degraded_v6/droso_aligned/counts/MV411_IRF2BP2_DMSO-H3K27AC_R1_T1_counts.txt
gs://amlproject/Chip/IRF2BP2_degraded_v6/droso_aligned/counts/MV411_IRF2BP2_DMSO-H3K27AC_R2_T1_counts.txt
gs://amlproject/Chip/IRF2BP2_degraded_v6/droso_aligned/counts/MV411_IRF2BP2_DMSO-MED1_R1_T1_counts.txt
gs://amlproject/Chip/IRF2BP2_degraded_v6/droso_aligned/counts/MV411_IRF2BP2_DMSO-MED1_R2_T1_counts.txt
gs://amlproject/Chip/IRF2BP2_degraded_v6/droso_aligned/counts/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_T1_counts.txt
gs://amlproject/Chip/IRF2BP2_degraded_v6/droso_aligned/counts/

In [19]:
counts = ! gsutil cat gs://amlproject/Chip/$project/droso_aligned/counts/*
samples = ! gsutil ls gs://amlproject/Chip/$project/droso_aligned/counts/

for c,s in zip(counts, samples): # get unique mapped reads(droso) to sample
    print(s.split("/")[-1]+" : "+c)

INPUT_R1_T1_counts.txt : 2923704
MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_T1_counts.txt : 1313814
MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_T1_counts.txt : 1475226
MV411_IRF2BP2_DMSO-H3K27AC_R1_T1_counts.txt : 2680444
MV411_IRF2BP2_DMSO-H3K27AC_R2_T1_counts.txt : 1429718
MV411_IRF2BP2_DMSO-MED1_R1_T1_counts.txt : 721324
MV411_IRF2BP2_DMSO-MED1_R2_T1_counts.txt : 1444440
MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_T1_counts.txt : 1713190
MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2_T1_counts.txt : 1024296
MV411_IRF2BP2_VHL-H3K27AC_R1_T1_counts.txt : 2024650
MV411_IRF2BP2_VHL-H3K27AC_R2_T1_counts.txt : 1891418
MV411_IRF2BP2_VHL-MED1_R1_T1_counts.txt : 1285272
MV411_IRF2BP2_VHL-MED1_R2_T1_counts.txt : 1107752


In [ ]:
# now use the total counts in gs://amlproject/Chip/IRF2BP2_degraded_v6/multiqc/broadPeak/multiqc_report.html
# use MERGED LIB: SAMTools (filtered) > Percent Mapped data [samtools_alignment_plot-2]
# https://console.cloud.google.com/storage/browser/amlproject/Chip/IRF2BP2_degraded_v6/multiqc/broadPeak/multiqc_report.html

In [20]:
! gsutil ls gs://amlproject/Chip/$project/multiqc/broadPeak/

gs://amlproject/Chip/IRF2BP2_degraded_v6/multiqc/broadPeak/multiqc_report.html


In [112]:
# copy bams, bigwigs, and macs data to local comp
h.createFoldersFor('../../data/'+project+'/bwa/mergedLibrary/')
! gsutil -m cp gs://amlproject/Chip/$project/bwa/mergedLibrary/*.ba* ../../data/$project/bwa/mergedLibrary/
! gsutil -m cp -r gs://amlproject/Chip/$project/bwa/mergedLibrary/bigwig/ ../../data/$project/bwa/mergedLibrary/
! gsutil -m cp -r gs://amlproject/Chip/$project/bwa/mergedLibrary/macs/ ../../data/$project/bwa/mergedLibrary/

Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam.bai...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/bwa/mergedLibrary/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/bwa/mergedLibrary/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1.mLb.clN.sorted.bam.bai...
Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/bwa/mergedLibrary/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/bwa/mergedLibrary/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2.mLb.clN.sorted.bam.bai...
Copying gs://amlproject/Chip/IRF2BP2_de

Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/consensus/MED1/MED1.consensus_peaks.boolean.intersect.txt...
Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/consensus/MED1/MED1.consensus_peaks.boolean.annotatePeaks.txt...
Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/consensus/MED1/MED1.consensus_peaks.boolean.txt...
Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/consensus/MED1/MED1.consensus_peaks.featureCounts.txt...
Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/consensus/MED1/MED1.consensus_peaks.featureCounts.txt.summary...
Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/qc/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/qc/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_peaks.FRiP_mqc.tsv...
Copyin

Computing scales from AML sample tracker excel sheet

In [34]:
scales = [ #same as spreadsheet order
    1.00, #MV411_IRF2BP2_DMSO FLAG_IRF2BP2 R1
    0.83, #MV411_IRF2BP2_DMSO FLAG_IRF2BP2 R2
    0.77, #MV411_IRF2BP2_DMSO H3K27AC R1
    1.00, #MV411_IRF2BP2_DMSO H3K27AC R2
    1.00, #MV411_IRF2BP2_DMSO MED1 R1
    0.91, #MV411_IRF2BP2_DMSO MED1 R2
    
    0.90, #MV411_IRF2BP2_VHL FLAG_IRF2BP2 R1
    1.00, #MV411_IRF2BP2_VHL FLAG_IRF2BP2 R2
    1.00, #MV411_IRF2BP2_VHL H3K27AC R1
    0.81, #MV411_IRF2BP2_VHL H3K27AC R2
    0.54, #MV411_IRF2BP2_VHL MED1 R1
    1.00, #MV411_IRF2BP2_VHL MED1 R2
    #1.00  #MV411 INPUT (SKIP)
]

In [9]:
bams = ! ls ../../data/$project/bwa/mergedLibrary/*.bam
bams_short = list(i.split("/")[6] for i in bams)
bams_short

['INPUT_R1.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_DMSO-H3K27AC_R1.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_DMSO-H3K27AC_R2.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_DMSO-MED1_R1.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_DMSO-MED1_R2.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_VHL-H3K27AC_R1.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_VHL-H3K27AC_R2.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_VHL-MED1_R1.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_VHL-MED1_R2.mLb.clN.sorted.bam']

In [10]:
names = ["FLAG_IRF2BP2_R1", "FLAG_IRF2BP2_R2", "H3K27AC_R1", "H3K27AC_R2", "MED1_R1", "MED1_R2"]

### on scalled data

In [29]:
! mkdir ../../data/$project/diffPeaks/ && ! mkdir ../../data/$project/diffData/

In [11]:
wigs = ! ls ../../data/$project/bwa/mergedLibrary/bigwig/*.bigWig
wigs

['../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/INPUT_R1.bigWig',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1.bigWig',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2.bigWig',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_DMSO-H3K27AC_R1.bigWig',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_DMSO-H3K27AC_R2.bigWig',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_DMSO-MED1_R1.bigWig',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_DMSO-MED1_R2.bigWig',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1.bigWig',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2.bigWig',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_VHL-H3K27AC_R1.bigWig',
 '../../data/IRF2BP2_degraded_v6/bwa/mer

In [12]:
bams_short

['INPUT_R1.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_DMSO-H3K27AC_R1.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_DMSO-H3K27AC_R2.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_DMSO-MED1_R1.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_DMSO-MED1_R2.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_VHL-H3K27AC_R1.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_VHL-H3K27AC_R2.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_VHL-MED1_R1.mLb.clN.sorted.bam',
 'MV411_IRF2BP2_VHL-MED1_R2.mLb.clN.sorted.bam']

#### diffPeak on scaled data (full reprocessing)

In [109]:
for i in range(int(len(bams)/2)):
    if i < 0:
        continue
    bam1 = bams[1+i]
    bam2 = bams[1+len(names)+i]
    print(bams_short[1+i])
    print(bams_short[1+len(names)+i])
    print(await chip.fullDiffPeak(bam1, bam2, control1 = bams[0], 
                            scaling = [scales[i], scales[len(names)+i]], 
                            directory = "../../data/"+project+"/diffData/", 
                            res_directory = "../../data/"+project+"/diffPeaks/", 
                            pairedend=True))
    print("\n")

MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1.mLb.clN.sorted.bam
MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1.mLb.clN.sorted.bam
doing diff from ../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1.mLb.clN.sorted.bam and ../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1.mLb.clN.sorted.bam
computing the fragment avg size
234
computing the scaling values
b'INFO  @ Thu, 22 Jul 2021 15:58:36: \n# Command line: callpeak -B -t ../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1.mLb.clN.sorted.bam -c ../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 234 -n MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1 --outdir ../../data/IRF2BP2_degraded_v6/diffData/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1\n# format = BAMPE\n# ChIP-seq file = [\'../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1.mLb.clN.sorted.bam\']\n# control file = [

CompletedProcess(args='macs3 bdgdiff --t1 ../../data/IRF2BP2_degraded_v6/diffData/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileup.bdg --c1 ../../data/IRF2BP2_degraded_v6/diffData/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_control_lambda.bdg --t2 ../../data/IRF2BP2_degraded_v6/diffData/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_treat_pileup.bdg --c2 ../../data/IRF2BP2_degraded_v6/diffData/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_control_lambda.bdg --d1 28332133 --d2 37081447 -g 60 -l 234 --o-prefix MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_treat_pileup --outdir ../../data/IRF2BP2_degraded_v6/diffPeaks/', returncode=0, stdout=b'', stderr=b'INFO  @ Thu, 22 Jul 2021 16:39:38: Read and build treatment 1 bedGraph... \nINFO  @ Thu, 22 Jul 2021 16:40:16: Read and build control 1 bedGraph... \nINFO  @ Thu, 22 Jul 2021 16:42:43: Read and build treatment 2 bedGraph... \nINFO  @ Thu, 22 Jul 2021 16:43:27: Read and build control 2 bedGraph... \nINFO  @ Thu, 22 Jul 2021 16:52:04: 

b'INFO  @ Thu, 22 Jul 2021 17:15:14: \n# Command line: callpeak -B -t ../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2.mLb.clN.sorted.bam -c ../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 225 -n MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2 --outdir ../../data/IRF2BP2_degraded_v6/diffData/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2\n# format = BAMPE\n# ChIP-seq file = [\'../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2.mLb.clN.sorted.bam\']\n# control file = [\'../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n# Larger dataset will be scaled towards sma

CompletedProcess(args='macs3 bdgdiff --t1 ../../data/IRF2BP2_degraded_v6/diffData/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_treat_pileup.bdg --c1 ../../data/IRF2BP2_degraded_v6/diffData/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_control_lambda.bdg --t2 ../../data/IRF2BP2_degraded_v6/diffData/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2_treat_pileup.bdg --c2 ../../data/IRF2BP2_degraded_v6/diffData/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2_control_lambda.bdg --d1 34242045 --d2 23693935 -g 60 -l 225 --o-prefix MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2_treat_pileup --outdir ../../data/IRF2BP2_degraded_v6/diffPeaks/', returncode=0, stdout=b'', stderr=b'INFO  @ Thu, 22 Jul 2021 17:36:00: Read and build treatment 1 bedGraph... \nINFO  @ Thu, 22 Jul 2021 17:36:40: Read and build control 1 bedGraph... \nINFO  @ Thu, 22 Jul 2021 17:39:10: Read and build treatment 2 bedGraph... \nINFO  @ Thu, 22 Jul 2021 17:39:42: Read and build control 2 bedGraph... \nINFO  @ Thu, 22 Jul 2021 17:48:16: 

IndexError: list index out of range

In [40]:
initscales = ! cat ../../data/$project/bwa/mergedLibrary/bigwig/scale/*.txt
initscales

['0.00744797',
 '0.017587',
 '0.0175161',
 '0.011023',
 '0.0118437',
 '0.0172945',
 '0.019515',
 '0.0149292',
 '0.0210329',
 '0.0111481',
 '0.0110806',
 '0.0183179',
 '0.0230248']

In [45]:
#! tail -n +1 ../../data/$project/bwa/mergedLibrary/bigwig/scale/*.txt
file_order = ! ls ../../data/$project/bwa/mergedLibrary/bigwig/scale/*.txt
file_order = [os.path.basename(f) for f in file_order]
file_order

['INPUT_R1.scale_factor.txt',
 'MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1.scale_factor.txt',
 'MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2.scale_factor.txt',
 'MV411_IRF2BP2_DMSO-H3K27AC_R1.scale_factor.txt',
 'MV411_IRF2BP2_DMSO-H3K27AC_R2.scale_factor.txt',
 'MV411_IRF2BP2_DMSO-MED1_R1.scale_factor.txt',
 'MV411_IRF2BP2_DMSO-MED1_R2.scale_factor.txt',
 'MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1.scale_factor.txt',
 'MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2.scale_factor.txt',
 'MV411_IRF2BP2_VHL-H3K27AC_R1.scale_factor.txt',
 'MV411_IRF2BP2_VHL-H3K27AC_R2.scale_factor.txt',
 'MV411_IRF2BP2_VHL-MED1_R1.scale_factor.txt',
 'MV411_IRF2BP2_VHL-MED1_R2.scale_factor.txt']

In [39]:
rescales = [val*float(initscales[1+i]) for i, val in enumerate(scales)]
rescales

[0.017587,
 0.014538362999999999,
 0.00848771,
 0.0118437,
 0.0172945,
 0.01775865,
 0.01343628,
 0.0210329,
 0.0111481,
 0.008975286,
 0.009891666000000002,
 0.0230248]

In [52]:
chip.bigWigFrom(bams[1:], 
                genome='GRCh38',scaling=rescales,
                numthreads=8)

In [54]:
! mkdir ../../data/$project/recalib_bigwig/

In [55]:
! mv bigwig/* ../../data/$project/recalib_bigwig/

In [56]:
! gsutil -m cp -r ../../data/$project/recalib_bigwig gs://amlproject/Chip/$project/

Copying file://../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_VHL-H3K27AC_R2.bw [Content-Type=application/octet-stream]...
Copying file://../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2.bw [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file://../../data/IRF2BP2_de

In [57]:
os.popen('for i in $(ls ../../data/'+project+'/diffPeaks/*.bed); \
            do echo $(wc -l $i); \
            done').read().split('\n')

['1959 ../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_treat_pileup_c3.0_common.bed',
 '45 ../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_treat_pileup_c3.0_cond1.bed',
 '645 ../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_treat_pileup_c3.0_cond2.bed',
 '1268 ../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2_treat_pileup_c3.0_common.bed',
 '2 ../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2_treat_pileup_c3.0_cond1.bed',
 '2661 ../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2_treat_pileup_c3.0_cond2.bed',
 '31743 ../../data/IRF2BP2

In [20]:
bed_counts = os.popen('for i in $(ls ../../data/'+project+'/diffPeaks/*.bed); \
            do echo $(wc -l $i); \
            done').read().split('\n')

In [22]:
len(bed_counts)

19

In [79]:
bed_counts = ! wc -l ../../data/$project/diffPeaks/*.bed
bed_counts

['    1959 ../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_treat_pileup_c3.0_common.bed',
 '      45 ../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_treat_pileup_c3.0_cond1.bed',
 '     645 ../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_treat_pileup_c3.0_cond2.bed',
 '    1268 ../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2_treat_pileup_c3.0_common.bed',
 '       2 ../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2_treat_pileup_c3.0_cond1.bed',
 '    2661 ../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2_treat_pileup_c3.0_cond2.be

In [68]:
def getDiffPeakCounts(bed_counts, scaled):
    names = ["FLAG_IRF2BP2_R1", "FLAG_IRF2BP2_R2", "H3K27AC_R1", "H3K27AC_R2", "MED1_R1", "MED1_R2"]
    peak_types = ["common", "cond1", "cond2"]
    rows = []
    
    for i,s in enumerate(bed_counts[:-1]):
        counts = re.search(r'\d+', s).group()
        file = s.split("    ")[-1].split("/")[-1]
        sample_group = next(name for name in names if name in file)
        cond_dict = {"common": "COMMON", "cond1": "DMSO", "cond2": "VHL"}
        peak_cond = cond_dict.get(next(peak_type for peak_type in peak_types if peak_type in file))
        rows.append([scaled, sample_group, peak_cond, file, counts])
    
    return(pd.DataFrame(rows, columns=["SCALE", "SAMPLE", "PEAK_TYPE", "FILE", "N_PEAK"]))

In [80]:
df_diff_peak = getDiffPeakCounts(bed_counts, scaled="SCALED")
df_diff_peak.head()

,SCALE,SAMPLE,PEAK_TYPE,FILE,N_PEAK
0,SCALED,FLAG_IRF2BP2_R1,COMMON,MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileu...,1959
1,SCALED,FLAG_IRF2BP2_R1,DMSO,MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileu...,45
2,SCALED,FLAG_IRF2BP2_R1,VHL,MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileu...,645
3,SCALED,FLAG_IRF2BP2_R2,COMMON,MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_treat_pileu...,1268
4,SCALED,FLAG_IRF2BP2_R2,DMSO,MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_treat_pileu...,2


In [58]:
bw = ! ls ../../data/$project/recalib_bigwig/*
bw

['../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_DMSO-H3K27AC_R1.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_DMSO-H3K27AC_R2.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_DMSO-MED1_R1.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_DMSO-MED1_R2.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_VHL-H3K27AC_R1.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_VHL-H3K27AC_R2.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_VHL-MED1_R1.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_VHL-MED1_R2.bw']

In [64]:
! mkdir ../results/$project/
! mkdir ../results/$project/plots
! mkdir ../results/$project/plots/heatmaps/

In [ ]:
#! gsutil -m cp gs://amlproject/Chip/$project/bwa/mergedLibrary/macs/broadPeak/*.broadPeak ../../data/$project/bwa/mergedLibrary/macs/broadPeak/

In [65]:
peaks = ! ls ../../data/$project/bwa/mergedLibrary/macs/broadPeak/*.broadPeak
peaks

['../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_peaks.broadPeak',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_peaks.broadPeak',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/MV411_IRF2BP2_DMSO-H3K27AC_R1_peaks.broadPeak',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/MV411_IRF2BP2_DMSO-H3K27AC_R2_peaks.broadPeak',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/MV411_IRF2BP2_DMSO-MED1_R1_peaks.broadPeak',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/MV411_IRF2BP2_DMSO-MED1_R2_peaks.broadPeak',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_peaks.broadPeak',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2_peaks.broadPeak',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/macs/broadPeak/MV411_IRF2

In [108]:
names = ["FLAG_IRF2BP2_R1", "FLAG_IRF2BP2_R2", "H3K27AC_R1", "H3K27AC_R2", "MED1_R1", "MED1_R2"]

#### merging peaks VHL/DMSO

In [71]:
mpeaks = []
for i, val in enumerate(names):
    if i<0:
        continue
    print(val)
    dmso = peaks[i]
    vhl = peaks[i+len(names)]
    chip.simpleMergePeaks(pd.concat([chip.loadPeaks(dmso), chip.loadPeaks(vhl)])).to_csv('../../data/' + project + '/' + val + '_genomewide_merged.bed', sep='\t', header=False, index=False)
    mpeaks.append('../../data/'+project+'/'+val+'_genomewide_merged.bed')

FLAG_IRF2BP2_R1


/home/monika/genepy/genepy/epigenetics/chipseq.py:127: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  bindings = bindings.drop(5, 1).drop(4, 1).rename(columns={6:4, 7:5, 8:6, 9:7,})


0.08421122374610747

0.38918704034983104

0.6834956602398463

0.7897700920956735

0.8869012124826078

0.9886702444841979

FLAG_IRF2BP2_R2813
H3K27AC_R1489355497654
H3K27AC_R2334389194557
MED1_R1156020704175555
MED1_R2382515149456456


In [70]:
for i, val in enumerate(names):
    if i<0:
        continue
    print(val)
    dmso = peaks[i]
    vhl = peaks[i+len(names)]
    print(os.path.basename(dmso))
    print(os.path.basename(vhl))
    print("\n")

FLAG_IRF2BP2_R1
MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_peaks.broadPeak
MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_peaks.broadPeak


FLAG_IRF2BP2_R2
MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_peaks.broadPeak
MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2_peaks.broadPeak


H3K27AC_R1
MV411_IRF2BP2_DMSO-H3K27AC_R1_peaks.broadPeak
MV411_IRF2BP2_VHL-H3K27AC_R1_peaks.broadPeak


H3K27AC_R2
MV411_IRF2BP2_DMSO-H3K27AC_R2_peaks.broadPeak
MV411_IRF2BP2_VHL-H3K27AC_R2_peaks.broadPeak


MED1_R1
MV411_IRF2BP2_DMSO-MED1_R1_peaks.broadPeak
MV411_IRF2BP2_VHL-MED1_R1_peaks.broadPeak


MED1_R2
MV411_IRF2BP2_DMSO-MED1_R2_peaks.broadPeak
MV411_IRF2BP2_VHL-MED1_R2_peaks.broadPeak




In [72]:
mpeaks

['../../data/IRF2BP2_degraded_v6/FLAG_IRF2BP2_R1_genomewide_merged.bed',
 '../../data/IRF2BP2_degraded_v6/FLAG_IRF2BP2_R2_genomewide_merged.bed',
 '../../data/IRF2BP2_degraded_v6/H3K27AC_R1_genomewide_merged.bed',
 '../../data/IRF2BP2_degraded_v6/H3K27AC_R2_genomewide_merged.bed',
 '../../data/IRF2BP2_degraded_v6/MED1_R1_genomewide_merged.bed',
 '../../data/IRF2BP2_degraded_v6/MED1_R2_genomewide_merged.bed']

#### GENOME WIDE comparison

In [80]:
dmsoname = "_DMSO"
condname = "_IRF2BP2_ko"

In [78]:
mpeaks = ! ls ../../data/$project/*_genomewide_merged.bed

../../data/IRF2BP2_degraded_v6/FLAG_IRF2BP2_R1_genomewide_merged.bed
../../data/IRF2BP2_degraded_v6/FLAG_IRF2BP2_R2_genomewide_merged.bed
../../data/IRF2BP2_degraded_v6/H3K27AC_R1_genomewide_merged.bed
../../data/IRF2BP2_degraded_v6/H3K27AC_R2_genomewide_merged.bed
../../data/IRF2BP2_degraded_v6/MED1_R1_genomewide_merged.bed
../../data/IRF2BP2_degraded_v6/MED1_R2_genomewide_merged.bed


In [81]:
bw

['../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_DMSO-H3K27AC_R1.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_DMSO-H3K27AC_R2.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_DMSO-MED1_R1.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_DMSO-MED1_R2.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_VHL-H3K27AC_R1.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_VHL-H3K27AC_R2.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_VHL-MED1_R1.bw',
 '../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_VHL-MED1_R2.bw']

In [100]:
from genepy.epigenetics.plot import getPeaksAt

In [101]:
for i, val in enumerate(bw):
    if i < 0:
        continue
    name = names[i-len(names)]+condname if i//len(names) else names[i]+dmsoname
    print(name)
    #print(mpeaks[i%len(names)])
    #print(val)
    #print("\n")
    genepyPlot.getPeaksAt(mpeaks[i%len(names)], bigwigs = val, bigwignames=name, 
               peaknames=['Macs2_Peaks'], window=3000, 
               folder="", title=name, numthreads=8, refpoint="center", 
               name='../../data/'+project+'/'+name+'_mat.pdf', 
               withDeeptools=True, torecompute=True, legendLoc="lower-left")

FLAG_IRF2BP2_R1_DMSO
CompletedProcess(args="computeMatrix reference-point -S ../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1.bw  --referencePoint center --regionsFileName ../../data/IRF2BP2_degraded_v6/FLAG_IRF2BP2_R1_genomewide_merged.bed  --missingDataAsZero --outFileName ../../data/IRF2BP2_degraded_v6/FLAG_IRF2BP2_R1_DMSO_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 8 && plotHeatmap --matrixFile ../../data/IRF2BP2_degraded_v6/FLAG_IRF2BP2_R1_DMSO_mat.gz --outFileName ../../data/IRF2BP2_degraded_v6/FLAG_IRF2BP2_R1_DMSO_mat.pdf --refPointLabel center --legendLocation lower-left --regionsLabel Macs2_Peaks --plotTitle 'FLAG_IRF2BP2_R1_DMSO'", returncode=0, stdout=b'', stderr=b'')
FLAG_IRF2BP2_R2_DMSO
CompletedProcess(args="computeMatrix reference-point -S ../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2.bw  --referencePoint center --regionsFileName ../../data/IRF2BP2_degraded_v6/FLAG_IRF2BP2_R2_genomewide_

#### making overlapping profiles

In [104]:
for i, val in enumerate(names):
    if i <0:
        continue
    val1 = '../../data/'+project+'/' + val + dmsoname+'_mat.gz'
    val2 = '../../data/'+project+'/' + val + condname+'_mat.gz'
    
    print(val)
    genepyPlot.makeProfiles(matx=[val1,val2], matnames=['DMSO','VHL'], title=val, 
                            refpoint="center", name='../../data/'+project+'/'+val+'_combined_mat.pdf', 
                            legendLoc="lower-left")

FLAG_IRF2BP2_R1
CompletedProcess(args='computeMatrixOperations relabel -m ../../data/IRF2BP2_degraded_v6/FLAG_IRF2BP2_R1_DMSO_mat.gz -o ../../data/IRF2BP2_degraded_v6/FLAG_IRF2BP2_R1_DMSO_mat.gz --groupLabels DMSO && computeMatrixOperations relabel -m ../../data/IRF2BP2_degraded_v6/FLAG_IRF2BP2_R1_IRF2BP2_ko_mat.gz -o ../../data/IRF2BP2_degraded_v6/FLAG_IRF2BP2_R1_IRF2BP2_ko_mat.gz --groupLabels VHL && computeMatrixOperations rbind -m ../../data/IRF2BP2_degraded_v6/FLAG_IRF2BP2_R1_DMSO_mat.gz ../../data/IRF2BP2_degraded_v6/FLAG_IRF2BP2_R1_IRF2BP2_ko_mat.gz -o ../../data/IRF2BP2_degraded_v6/FLAG_IRF2BP2_R1_combined_mat.gz && plotProfile --matrixFile ../../data/IRF2BP2_degraded_v6/FLAG_IRF2BP2_R1_combined_mat.gz --outFileName ../../data/IRF2BP2_degraded_v6/FLAG_IRF2BP2_R1_combined_mat.pdf --refPointLabel center --legendLocation lower-left --plotTitle FLAG_IRF2BP2_R1', returncode=0, stdout=b'', stderr=b'')
FLAG_IRF2BP2_R2
CompletedProcess(args='computeMatrixOperations relabel -m ../../dat

In [105]:
h.createFoldersFor('../results/'+project+'/plots/scaled/heatmaps/')
! cp ../../data/$project/*.pdf ../results/$project/plots/scaled/heatmaps/

cp: cannot stat '../../data/IRF2BP2_degraded_v6/diffPeaks/*.pdf': No such file or directory


In [106]:
cond1peak = ! ls ../../data/$project/diffPeaks/*cond1.bed
cond2peak = ! ls ../../data/$project/diffPeaks/*cond2.bed
commonpeak = ! ls ../../data/$project/diffPeaks/*common.bed
cond1peak

['../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_treat_pileup_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2_treat_pileup_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-H3K27AC_R1_treat_pileup_vs_MV411_IRF2BP2_VHL-H3K27AC_R1_treat_pileup_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-H3K27AC_R2_treat_pileup_vs_MV411_IRF2BP2_VHL-H3K27AC_R2_treat_pileup_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-MED1_R1_treat_pileup_vs_MV411_IRF2BP2_VHL-MED1_R1_treat_pileup_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-MED1_R2_treat_pileup_vs_MV411_IRF2BP2_VHL-MED1_R2_treat_pileup_c3.0_cond1.bed']

In [109]:
for i in range(int(len(bw)/2)):
    if i <0:
        continue
    name1 = bw[i]
    name2 = bw[i+len(names)]
    peak = [cond1peak[i], commonpeak[i], cond2peak[i]]
   # for val in peak:
      #  chip.dropWeirdChromosomes(val)
    name = names[i]
    print(str(i+1)+": "+name)
    print(name1,name2)
    genepyPlot.getPeaksAt(peak, [name1, name2], bigwignames=['DMSO', 'VHL'], 
                    peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], 
                    window=3000, folder="", title=name, numthreads=8, refpoint="center", 
                    name='../../data/'+project+'/diffPeaks/'+name+'_mat.pdf', 
                    withDeeptools=True, torecompute=False)

1: FLAG_IRF2BP2_R1
../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1.bw ../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1.bw
CompletedProcess(args="computeMatrix reference-point -S ../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1.bw ../../data/IRF2BP2_degraded_v6/recalib_bigwig/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1.bw  --referencePoint center --regionsFileName ../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_treat_pileup_c3.0_cond1.bed ../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_treat_pileup_c3.0_common.bed ../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_treat_pileup_c3.0_cond2.bed  --missingDataAsZero --outFileName ../../data/IRF2BP2_degraded_v6/diffPeaks/FLAG_IRF2

In [110]:
! mkdir ../results/$project/plots/scaled/diffPeaks/
! cp ../../data/$project/diffPeaks/*.pdf ../results/$project/plots/scaled/diffPeaks/

mkdir: cannot create directory ‘../results/IRF2BP2_degraded_v6/plots/scaled/diffPeaks/’: File exists


#### looking at TSS

In [ ]:
indextarget = 
targetFile = '../results/slamseqMax/IRF8targets.txt' #???
if targetFile && indextarget:
IRF8targets = h.fileToList(targetFile)

### on unscalled data

In [111]:
! mkdir ../../data/$project/diffPeaks_unscaled

mkdir: cannot create directory ‘../../data/IRF2BP2_degraded_v6/diffPeaks_unscaled’: File exists


#### diffPeak on unscaled data (full reprocessing)

In [78]:
# diffPeak on unscaled data
for i in range(int(len(bams)/2)):
    if i < 0:
        continue
    print("Running sample group: {}".format(i))
    bam1 = bams[1+i]
    bam2 = bams[1+len(names)+i]
    print(bams_short[1+i])
    print(bams_short[1+len(names)+i])
    print(await chip.fullDiffPeak(bam1, bam2, control1 = bams[0], 
                            scaling = None, 
                            directory = "../../data/"+project+"/diffData_unscaled/", 
                            res_directory = "../../data/"+project+"/diffPeaks_unscaled/", 
                            pairedend=True))
    print("\n")

NameError: name 'bams' is not defined

In [81]:
bed_counts_unscaled = ! wc -l ../../data/$project/diffPeaks_unscaled/*.bed
df_diff_peak = df_diff_peak.append(getDiffPeakCounts(bed_counts_unscaled, scaled="UNSCALED"), ignore_index=True)
df_diff_peak

,SCALE,SAMPLE,PEAK_TYPE,FILE,N_PEAK
0,SCALED,FLAG_IRF2BP2_R1,COMMON,MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileu...,1959
1,SCALED,FLAG_IRF2BP2_R1,DMSO,MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileu...,45
2,SCALED,FLAG_IRF2BP2_R1,VHL,MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileu...,645
3,SCALED,FLAG_IRF2BP2_R2,COMMON,MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_treat_pileu...,1268
4,SCALED,FLAG_IRF2BP2_R2,DMSO,MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_treat_pileu...,2
5,SCALED,FLAG_IRF2BP2_R2,VHL,MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_treat_pileu...,2661
6,SCALED,H3K27AC_R1,COMMON,MV411_IRF2BP2_DMSO-H3K27AC_R1_treat_pileup_vs_...,31743
7,SCALED,H3K27AC_R1,DMSO,MV411_IRF2BP2_DMSO-H3K27AC_R1_treat_pileup_vs_...,508
8,SCALED,H3K27AC_R1,VHL,MV411_IRF2BP2_DMSO-H3K27AC_R1_treat_pileup_vs_...,2245
9,SCALED,H3K27AC_R2,COMMON,MV411_IRF2BP2_DMSO-H3K27AC_R2_treat_pileup_vs_...,30990


In [82]:
# diff peak count CSV
diff_peak_count_file = "../../data/"+project+"/"+project+"_diff_peak_counts.csv"
diff_peak_count_file_path = os.path.abspath(diff_peak_count_file)
print(diff_peak_count_file_path)
df_diff_peak.to_csv(diff_peak_count_file_path, index=False)

/home/monika/data/IRF2BP2_degraded_v6/IRF2BP2_degraded_v6_diff_peak_counts.csv


In [112]:
bw = ! ls ../../data/$project/bwa/mergedLibrary/bigwig/*.bigWig
bw

['../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/INPUT_R1.bigWig',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1.bigWig',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2.bigWig',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_DMSO-H3K27AC_R1.bigWig',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_DMSO-H3K27AC_R2.bigWig',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_DMSO-MED1_R1.bigWig',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_DMSO-MED1_R2.bigWig',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1.bigWig',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2.bigWig',
 '../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_VHL-H3K27AC_R1.bigWig',
 '../../data/IRF2BP2_degraded_v6/bwa/mer

#### making overlapping profiles

In [113]:
cond1peak = ! ls ../../data/$project/diffPeaks_unscaled/*cond1.bed
cond2peak = ! ls ../../data/$project/diffPeaks_unscaled/*cond2.bed
commonpeak = ! ls ../../data/$project/diffPeaks_unscaled/*common.bed
commonpeak

['../../data/IRF2BP2_degraded_v6/diffPeaks_unscaled/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_treat_pileup_c3.0_common.bed',
 '../../data/IRF2BP2_degraded_v6/diffPeaks_unscaled/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2_treat_pileup_c3.0_common.bed',
 '../../data/IRF2BP2_degraded_v6/diffPeaks_unscaled/MV411_IRF2BP2_DMSO-H3K27AC_R1_treat_pileup_vs_MV411_IRF2BP2_VHL-H3K27AC_R1_treat_pileup_c3.0_common.bed',
 '../../data/IRF2BP2_degraded_v6/diffPeaks_unscaled/MV411_IRF2BP2_DMSO-H3K27AC_R2_treat_pileup_vs_MV411_IRF2BP2_VHL-H3K27AC_R2_treat_pileup_c3.0_common.bed',
 '../../data/IRF2BP2_degraded_v6/diffPeaks_unscaled/MV411_IRF2BP2_DMSO-MED1_R1_treat_pileup_vs_MV411_IRF2BP2_VHL-MED1_R1_treat_pileup_c3.0_common.bed',
 '../../data/IRF2BP2_degraded_v6/diffPeaks_unscaled/MV411_IRF2BP2_DMSO-MED1_R2_treat_pileup_vs_MV411_IRF2BP2_VHL-MED1_R2_treat_pileup_c3.0_common.bed']

In [123]:
for i in range(int((len(bw)-1)/2)):
    #if i <0:
    #    continue
    name1 = bw[1+i]
    name2 = bw[1+len(names)+1]
    peak = [cond1peak[i], commonpeak[i], cond2peak[i]]
   # for val in peak:
      #  chip.dropWeirdChromosomes(val)
    name = names[i]
    print(str(i+1)+": "+name)
    #print(name)
    print(name1)
    print(name2)
    #print(peak)
    #print("\n")
    genepyPlot.getPeaksAt(peak, [name1, name2], bigwignames=['DMSO', 'VHL'], 
                          peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], window=3000, 
                          folder="", title=name, numthreads=8, refpoint="center", 
                          name='../../data/'+project+'/diffPeaks_unscaled/'+name+'_mat.pdf', 
                          withDeeptools=True)

1: FLAG_IRF2BP2_R1
../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1.bigWig
../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2.bigWig
CompletedProcess(args="computeMatrix reference-point -S ../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1.bigWig ../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2.bigWig  --referencePoint center --regionsFileName ../../data/IRF2BP2_degraded_v6/diffPeaks_unscaled/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_treat_pileup_c3.0_cond1.bed ../../data/IRF2BP2_degraded_v6/diffPeaks_unscaled/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_treat_pileup_c3.0_common.bed ../../data/IRF2BP2_degraded_v6/diffPeaks_unscaled/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_treat_pileup_c3.0_cond2.bed  -

CompletedProcess(args="computeMatrix reference-point -S ../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_DMSO-MED1_R2.bigWig ../../data/IRF2BP2_degraded_v6/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2.bigWig  --referencePoint center --regionsFileName ../../data/IRF2BP2_degraded_v6/diffPeaks_unscaled/MV411_IRF2BP2_DMSO-MED1_R2_treat_pileup_vs_MV411_IRF2BP2_VHL-MED1_R2_treat_pileup_c3.0_cond1.bed ../../data/IRF2BP2_degraded_v6/diffPeaks_unscaled/MV411_IRF2BP2_DMSO-MED1_R2_treat_pileup_vs_MV411_IRF2BP2_VHL-MED1_R2_treat_pileup_c3.0_common.bed ../../data/IRF2BP2_degraded_v6/diffPeaks_unscaled/MV411_IRF2BP2_DMSO-MED1_R2_treat_pileup_vs_MV411_IRF2BP2_VHL-MED1_R2_treat_pileup_c3.0_cond2.bed  --missingDataAsZero --outFileName ../../data/IRF2BP2_degraded_v6/diffPeaks_unscaled/MED1_R2_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 8 && plotHeatmap --matrixFile ../../data/IRF2BP2_degraded_v6/diffPeaks_unscaled/MED1_R2_mat.gz --outFileName ../../da

In [124]:
h.createFoldersFor('../results/'+project+'/plots/unscaled/diffPeaks/')
! cp ../../data/$project/diffPeaks_unscaled/*.pdf ../results/$project/plots/unscaled/diffPeaks/

! gsutil -m cp gs://amlproject/Chip/$project/bwa/mergedLibrary/deepTools/**.pdf ../results/$project/plots/

Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1.plotFingerprint.pdf...
Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2.plotFingerprint.pdf...
Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_IRF2BP2_DMSO-H3K27AC_R1.plotFingerprint.pdf...
Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_IRF2BP2_DMSO-H3K27AC_R2.plotFingerprint.pdf...
Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_IRF2BP2_DMSO-MED1_R1.plotFingerprint.pdf...
Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_IRF2BP2_DMSO-MED1_R2.plotFingerprint.pdf...
Copying gs://amlproject/Chip/IRF2BP2_degraded_v6/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_IRF2BP2_VHL-FLAG_IRF2

In [125]:
! gsutil -m cp -r ../../data/$project/diffPeaks gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/diffData gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/diffPeaks_unscaled gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/diffData_unscaled gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/peakplot gs://amlproject/Chip/$project/

Copying file://../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-MED1_R2_treat_pileup_vs_MV411_IRF2BP2_VHL-MED1_R2_treat_pileup_c3.0_common.bed [Content-Type=application/octet-stream]...
Copying file://../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-H3K27AC_R1_treat_pileup_vs_MV411_IRF2BP2_VHL-H3K27AC_R1_treat_pileup_c3.0_cond1.bed [Content-Type=application/octet-stream]...
Copying file://../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R2_treat_pileup_vs_MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R2_treat_pileup_c3.0_cond1.bed [Content-Type=application/octet-stream]...
Copying file://../../data/IRF2BP2_degraded_v6/diffPeaks/FLAG_IRF2BP2_R1_mat.pdf [Content-Type=application/pdf]...
Copying file://../../data/IRF2BP2_degraded_v6/diffPeaks/MV411_IRF2BP2_DMSO-H3K27AC_R2_treat_pileup_vs_MV411_IRF2BP2_VHL-H3K27AC_R2_treat_pileup_c3.0_cond2.bed [Content-Type=application/octet-stream]...
Copying file://../../data/IRF2BP2_degraded_v6/diffPeaks/FLAG_IRF2BP2_R1_m

Copying file://../../data/IRF2BP2_degraded_v6/diffData/MV411_IRF2BP2_VHL-FLAG_IRF2BP2_R1_summits.bed [Content-Type=application/octet-stream]...
Copying file://../../data/IRF2BP2_degraded_v6/diffData/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_peaks.xls [Content-Type=application/vnd.ms-excel]...
Copying file://../../data/IRF2BP2_degraded_v6/diffData/MV411_IRF2BP2_DMSO-H3K27AC_R2_summits.bed [Content-Type=application/octet-stream]...
Copying file://../../data/IRF2BP2_degraded_v6/diffData/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_summits.bed [Content-Type=application/octet-stream]...
Copying file://../../data/IRF2BP2_degraded_v6/diffData/MV411_IRF2BP2_VHL-MED1_R2_peaks.xls [Content-Type=application/vnd.ms-excel]...
Copying file://../../data/IRF2BP2_degraded_v6/diffData/MV411_IRF2BP2_VHL-H3K27AC_R1_summits.bed [Content-Type=application/octet-stream]...
Copying file://../../data/IRF2BP2_degraded_v6/diffData/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_control_lambda.bdg [Content-Type=application/octet-stream]...
Cop

\ [30/30 files][150.6 MiB/150.6 MiB] 100% Done                                  
Operation completed over 30 objects/150.6 MiB.                                   
Copying file://../../data/IRF2BP2_degraded_v6/diffData_unscaled/MV411_IRF2BP2_DMSO-MED1_R2_treat_pileup.bdg [Content-Type=application/octet-stream]...
Copying file://../../data/IRF2BP2_degraded_v6/diffData_unscaled/MV411_IRF2BP2_DMSO-FLAG_IRF2BP2_R1_treat_pileup.bdg [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help 